# Time Evolution OF A Quantum wavepacket under Harmonic Potential

# Solving the 1-D Schrodinger Equation
$$ i \hbar \frac{\partial \psi}{\partial t} = -\frac{\hbar^2}{2m} \frac{\partial^2\psi}{\partial x^2} + V(x)\psi $$

## Discretization (after dimensional analysis)

$$ i \frac{d\phi}{dt} = H \phi $$

where

$$H_{ij} = -\frac{1}{\Delta^2}\left(\delta_{i+1,j}-2\delta_{i,j}+\delta_{i-1,j}\right) + U_i \delta_{i,j} $$  


Formal solution:  

$$ \phi(t) = e^{-i H t} \phi(0) $$

Strategy:  
1. Solve for eigenvalues and eigenvectors of $H$  

$$ H \phi_n = \epsilon_n \phi_n $$  

2. Expand $\phi(0)$ in the basis of eigenvectors  

$$ \phi(0) = \sum_n c_n \phi_n $$  
where $c_n = \phi_n^{\dagger} \phi(0)$  

3. The state $\phi(t)$ is then given by  

\begin{eqnarray*}
\phi(t) &=& e^{-i H t} \phi(0) \\
&=& \sum_n c_n e^{-i\epsilon_n t}\phi_n
\end{eqnarray*}

In [1]:
##importing libraries
import numpy as np
from ipywidgets import*
from bqplot import pyplot as plt

**Defining Functions**
* First function will calculate $\phi \phi^{*} $
* Second one will return Gaussian function

In [2]:
def prob(H): ##probabilty function
    bn = np.conjugate(H)   
    vb = np.multiply(bn,H)
    return np.real(vb);

N = 400   ##number of descretee space points
sig = 1  ##width of gaussian 
peak = 0.  ## value at which Gaussian exhibits its peak
xs = np.linspace(-10,10,N)  ## creating space arrya 
dx = xs[4]-xs[3] ##finding delta x

k0 = 1.4 ## to provide momentum to the wavefunction 

def phi(x,sig,peak):
    const = 1.0/(np.sqrt(2*np.pi*sig*sig))
    return const*(np.exp(-((x-peak)**2)/(2*sig**2)))*np.exp(-1.j*k0*x)


def V0(c):   ###HARMONIC POTENTIAL
    return (1.0/2)*c*c;

**Defining parameters and Normalized Gaussian Function**

In [3]:
Psi = np.array([phi(x,sig,peak) for x in xs],dtype = complex)  ##gaussian function
norm_const = 1.0/np.sqrt(np.sum(prob(Psi)))  ##normalization constant
Psi = norm_const*Psi.reshape(N,1)  ## Normalized Wave function it's a column vector



Vpo = np.array([V0(xc) for xc in xs])
fig = plt.figure(layout= {'width':'860px','height':'600px'}) ##initializing figures
plt.clear() ## 

Psi_sqr = prob(Psi) ## probability array beacuse it is only meaningful 

fig = plt.figure(layout= {'width':'860px','height':'670px'})

plt.clear()

s_p = 1000
plot = plt.plot(xs,s_p*Psi_sqr,colors = ['orange']) ##I have multiply the psi square array by s_p to scale it
plt.plot(xs,Vpo,colors = ['black'])
plt.show()

## **TIME EVOLUTION (MAIN PART)**

In [4]:
Psi_History = Psi_sqr.reshape(N,1)  ##we will store Psi_square value at different instant in this array as a column matrix which will be accesed later for animation

def kdirac(i,j): ##kronecker delta function
    if(i==j):
        return 1;
    else:
        return 0;
def hamil(i,j): ##hamiltonian function
    v =(-kdirac(i+1,j)+2*kdirac(i,j)-kdirac(i-1,j))/(dx**2) + kdirac(i,j)*Vpo[i] ;
    return v;

Ham = np.zeros(N*N).reshape(N,N)   #initializing hamiltonian matrrix


for i in range(N):  #populating hamiltonian matrix
    for j in range(N):
        Ham[i,j] = hamil(i,j)
Eigval,Eigvect = np.linalg.eigh(Ham)      # calculataing eigenvectors and eigenvalues


## Eigen vectors contains all the wave vector, and these are orthonormal (check it!)
## and we have to find the coefficient lamda so we must have a matrix that will contain 
## hermitian conjugate of each wavevector i.e.convertinng them in a row then taking it's complex conjugate
##well this can be done in one line only :) using np.conjugate


conjugates = np.transpose(np.conjugate(Eigvect)) ##this will just take the complex conjugate of eeach vector 


Eigvect = np.transpose(Eigvect)   ## i did it because i have lambda as a column vectors so for easiness 

lambda_0 =  np.matmul(conjugates,Psi)  ## finding lambdas at t=0


lambda_0 = lambda_0.reshape(N,1) ### as i told you i am habitual to work with column matrix
Eigval = Eigval.reshape(N,1)

t= 0   # initial time 
dt = 0.1 ##time step

N_iter = 500  #number of iteration
for i in range(N_iter):
 

    lambda_t = lambda_0*np.exp(-1.j*Eigval*t) ##calculatating coefficient at later instant

    
    Psi_t = np.multiply(Eigvect,lambda_t)   ##finding psi(t) using superposition principal
    Psi_t = np.sum(Psi_t,axis = 0) ##taking sum along rows

        
    Psi_t = prob(Psi_t) 
    Psi_t = Psi_t.reshape(N,1)   ##column matrix :(
    Psi_History = np.hstack((Psi_History,Psi_t))  ##storing mod psi square
    t+= dt  ##updating time
    
    
    
    if(i%100==0 and i<=1000):
        
        print("|Phi|^2 = ",np.sum(Psi_sqr))

     

|Phi|^2 =  0.9999999999999996
|Phi|^2 =  0.9999999999999996
|Phi|^2 =  0.9999999999999996
|Phi|^2 =  0.9999999999999996
|Phi|^2 =  0.9999999999999996


## **SIMULATION**

In [5]:
animate_time = 80  ##you can change this time to tinker with animation
fig.animation_duration = animate_time-5  ##synchronizing figure with animation time 


def animate(N1):  ##animation function. widget will call it and it will show us what is mod psi square at later instant
    plot.y = s_p*Psi_History[:,N1]  

display(fig) ##displaying figure

linked_wdgt = interactive(animate,N1= Play(interval = animate_time,value=0,min=0,max=N_iter,step=1,descrption= "press play",disabled = False))

display(linked_wdgt)

Figure(animation_duration=75, axes=[Axis(scale=LinearScale()), Axis(orientation='vertical', scale=LinearScale(…

interactive(children=(Play(value=0, description='N1', interval=80, max=500), Output()), _dom_classes=('widget-…